# 수정한 점

1. hidden layer 3으로 조정
2. Hidden unit : 256
3. lr=0.001 / 학습 중 Learning Rate Scheduler를 사용해 점진적으로 감소시킴




## 1. STL10 데이터셋 클래스 정의
STL10 데이터셋을 학습 및 테스트 데이터로 로드하고, 파이토치 데이터로더를 통해 배치 단위로 데이터를 준비하는 작업을 수행   
해당 데이터셋을 모댈 학습에 사용할 수 있도록 전처리와 로더 생성까지 포함

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split


# 데이터 변환 정의
transform = transforms.Compose([
    # 데이터 증강
    transforms.RandomHorizontalFlip(),  # 랜덤 가로 뒤집기
    transforms.RandomRotation(10),     # 랜덤 회전


    transforms.ToTensor(),       # 이미지를 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화
])

# STL10 학습 및 테스트 데이터셋 다운로드
train_dataset = datasets.STL10(root='./data', split='train', transform=transform, download=True)
test_dataset = datasets.STL10(root='./data', split='test', transform=transform, download=True)

# Training 데이터를 Validation Set으로 나누기
train_indices, val_indices = train_test_split(range(len(train_dataset)), test_size=0.2, random_state=42) # Training 데이터의 일부를 Validation Set으로 나누어 모델 성능을 지속적으로 평가
train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(train_dataset, val_indices)

# DataLoader 생성 (64개의 배치로 나누고, 학습 데이터를 셔플)
batch_size = 64
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 2.64G/2.64G [04:54<00:00, 8.95MB/s]


Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


In [5]:
from torch import nn

# MLP 모델 정의
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers, output_size, activation_function):
        super(MLP, self).__init__()

        layers = []
        previous_size = input_size

        for _ in range(n_layers):
            layers.append(nn.Linear(previous_size, hidden_size))
            layers.append(nn.BatchNorm1d(hidden_size))  # 배치 정규화
            layers.append(activation_function())
            layers.append(nn.Dropout(0.3))  # 드롭아웃 확률 0.3으로 감소함
            previous_size = hidden_size

        layers.append(nn.Linear(previous_size, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# STL10 이미지 크기: 96x96, 채널 3 (3채널 이미지를 펼침)
input_size = 96 * 96 * 3
hidden_size = 256 # hidden unit의 개수를 128에서 점점 줄여서 나가도 됨 -- 만약 줄인다면--> 특징 추출을 세밀화
n_layers = 3  # Hidden layer 개수
output_size = 10  # 클래스 개수


activation_function = nn.LeakyReLU # 활성화 함수 변경함

model = MLP(input_size, hidden_size, n_layers, output_size, activation_function)

In [12]:
from torch import optim
from torch.optim.lr_scheduler import StepLR

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# Optimizer 설계
lr = 0.001

# 에포크 설정
epochs = 100

# 옵티마이저와 스케줄러
optimizer = optim.Adam(model.parameters(), lr)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5) # # 매 20 에포크마다 학습률을 반으로 줄임


# epoch 별 train loss / validation loss / accuracy loss 저장할 list --> 매번 출력하여 추이 파악
list_epoch = []
list_train_loss = []
list_val_loss = []
list_acc = []
list_acc_epoch = []




# Train
# 학습 함수 정의
def train_model(model, dataloader, val_loader, criterion, optimizer, epochs, patience=4):
    # train_losses = []  # Store losses for each epoch
    # val_losses = []  # Store validation losses for each epoch
    # accuracies = []  # Store accuracies for each epoch
    best_val_loss = float('inf')  # 초기 최상의 val_loss를 무한대로 설정
    patience_counter = 0  # Early Stopping 카운터 초기화

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in dataloader:

            images = images.view(images.size(0), -1)  # 이미지를 1D 벡터로 변환
            optimizer.zero_grad()
            outputs = model(images)  # 모델에 입력

            loss = criterion(outputs, labels)  # 손실 계산
            loss.backward()  # 역전파
            optimizer.step()  # 파라미터 업데이트
            train_loss = train_loss + loss.item()

        train_loss = train_loss / len(dataloader)
        list_train_loss.append(train_loss)
        list_epoch.append(epoch)

        # validation loss
        val_loss = evaluate_loss(model, val_loader, criterion)
        list_val_loss.append(val_loss)

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Early Stopping 조건 확인 - val_loss가 증가하면 더이상 학습을 하지 않는 로직
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0  # 개선이 있으면 카운터 리셋
        else:
            patience_counter += 1  # 개선되지 않으면 카운터 증가
            print(f"Early Stopping Counter: {patience_counter}/{patience}")

            if patience_counter >= patience:  # patience에 도달하면 학습 종료
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

        # 스케줄러 업데이트
        scheduler.step()


# loss function 평가
def evaluate_loss(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.view(images.size(0), -1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)



# Evaluation
# 평가 함수 정의
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # 평가 시에는 그래디언트 계산 비활성화
        for images, labels in dataloader:
            images = images.view(images.size(0), -1)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # 예측 값 추출
            correct = correct + (predicted == labels).sum().item()
            total = total + labels.size(0)
    accuracy = correct / total
    list_acc.append(accuracy)
    list_acc_epoch.append(len(list_epoch))
    print(f"Accuracy: {accuracy * 100:.2f}%")
    return accuracy


In [10]:
# 모델 학습
train_model(model, train_loader, val_loader, criterion, optimizer, epochs)

Epoch [1/100], Train Loss: 1.7892, Val Loss: 1.7173
Epoch [2/100], Train Loss: 1.7106, Val Loss: 1.6819
Epoch [3/100], Train Loss: 1.6651, Val Loss: 1.6801
Epoch [4/100], Train Loss: 1.6203, Val Loss: 1.6594
Epoch [5/100], Train Loss: 1.5772, Val Loss: 1.6559
Epoch [6/100], Train Loss: 1.5592, Val Loss: 1.6596
Epoch [7/100], Train Loss: 1.5455, Val Loss: 1.6197
Epoch [8/100], Train Loss: 1.4927, Val Loss: 1.5988
Epoch [9/100], Train Loss: 1.4681, Val Loss: 1.5952
Epoch [10/100], Train Loss: 1.4493, Val Loss: 1.5816
Epoch [11/100], Train Loss: 1.4305, Val Loss: 1.5874
Epoch [12/100], Train Loss: 1.4231, Val Loss: 1.5679
Epoch [13/100], Train Loss: 1.3971, Val Loss: 1.5685
Epoch [14/100], Train Loss: 1.3801, Val Loss: 1.5886
Epoch [15/100], Train Loss: 1.3524, Val Loss: 1.6334
Epoch [16/100], Train Loss: 1.3498, Val Loss: 1.5626
Epoch [17/100], Train Loss: 1.3425, Val Loss: 1.5637
Epoch [18/100], Train Loss: 1.3315, Val Loss: 1.5625
Epoch [19/100], Train Loss: 1.3095, Val Loss: 1.5368
Ep

In [11]:
# 모델 평가
accuracy = evaluate_model(model, test_loader)

Accuracy: 48.89%


# 결과물 기록
hidden_size = 256 / hidden layer = 3 / lr = 0.001 / 100 epoch / Act functino : LeakyReLU / Dropout : 0.3 / scheduler = StepLR(optimizer, step_size=20, gamma=0.5)- Accuracy: 48.89%

# 피드백

train loss는 에포크 늘어나면 무조건 줄으니까 train loss가 줄어든다고 해서 좋은 게 아니고 val loss를 잘 보자. 그리고 early stopping 추가하여 이상한 길로 가는 것 같으면 학습을 중단하자.

# 우성님 지원님 피드백
파라미터가 너무 적음
27,648(96x96x3)에서 256(hiddenUnit 설정 값)로 추출하고 있는데, 이는 시작부터 고된 추출일수도 있음을 인지하기. 그래서 모델의 파라미터 규모를 키우는 방법도 생각해보세요.
